### 簇内政策共识文件生成

In [4]:
import pandas as pd
from collections import Counter

df_policy = pd.read_csv("../data/3-2-Human_Recognition_Mode_Countries.csv", encoding='utf-8-sig')
df_mapping = pd.read_csv("../data/4-2-Consensus_Policy_Cluster_Mapping.csv", encoding='utf-8-sig')

# 构建国家-政策特征字典
country_policy_features = {}
for _, row in df_policy.iterrows():
    c, l2 = row['国家'], row['L2政策']
    country_policy_features.setdefault(c, {})[l2] = {
        'L1分类': row['L1分类'],
        'L1分类中文名': row['L1分类中文名'],
        'L2政策中文名': row['L2政策中文名'],
        '聚类ID': row['聚类ID'],
        'Starting': row['Starting'],
        'Trend': row['Trend'],
        'Ending': row['Ending']
    }

# 获取所有L2政策列表
l2_categories = df_policy['L2政策'].unique().tolist()

# 核心函数：计算簇内政策共识
def calc_cluster_consensus(cluster_countries, l2_policy):
    """统计簇内所有国家在指定政策上的(聚类ID,Starting,Trend,Ending)组合分布"""
    combos = []
    l1_value = None
    l1_name = None
    l2_name = None
    
    for c in cluster_countries:
        feats = country_policy_features.get(c, {}).get(l2_policy)
        if feats:
            combos.append((feats['聚类ID'], feats['Starting'], feats['Trend'], feats['Ending']))
            if l1_value is None:
                l1_value = feats['L1分类']
                l1_name = feats['L1分类中文名']
                l2_name = feats['L2政策中文名']
    
    if not combos:
        return []
    
    combo_counts = Counter(combos)
    n_total = len(cluster_countries)
    
    results = []
    for (cluster_id, start, trend, end), count in combo_counts.items():
        ratio = count / n_total
        results.append({
            'L1分类': l1_value,
            'L1分类中文名': l1_name,
            'L2政策中文名': l2_name,
            '政策聚类ID': cluster_id,
            'Starting': start,
            'Trend': trend,
            'Ending': end,
            '政策国家数': count,
            '共识占比': ratio
        })
    
    results.sort(key=lambda x: x['政策国家数'], reverse=True)
    return results

# 生成所有K值的共识记录
records = []

for k_value in df_mapping['K值'].unique():
    df_k = df_mapping[df_mapping['K值'] == k_value]
    
    for cluster_id in df_k['共识聚类ID'].unique():
        cluster_countries = df_k[df_k['共识聚类ID'] == cluster_id]['国家'].tolist()
        n_total = len(cluster_countries)
        
        for l2 in l2_categories:
            consensus_list = calc_cluster_consensus(cluster_countries, l2)
            
            for item in consensus_list:
                records.append({
                    'K值': k_value,
                    '共识聚类ID': cluster_id,
                    '簇内国家数': n_total,
                    'L2共识政策': l2,
                    'L2共识政策中文名': item['L2政策中文名'],
                    'L1分类': item['L1分类'],
                    'L1分类中文名': item['L1分类中文名'],
                    '政策聚类ID': item['政策聚类ID'],
                    'Starting': item['Starting'],
                    'Trend': item['Trend'],
                    'Ending': item['Ending'],
                    '政策国家数': item['政策国家数'],
                    '共识占比': item['共识占比']
                })

# 导出结果
df_result = pd.DataFrame(records)

if len(df_result) > 0:
    output_path = "../data/4-3-Per-Policy_Intra-Cluster_Consensus_Analysis.csv"
    df_result.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"输出文件：{output_path}")
    print(f"共生成 {len(df_result)} 条政策共识记录")


输出文件：../data/4-3-Per-Policy_Intra-Cluster_Consensus_Analysis.csv
共生成 4919 条政策共识记录


### 簇内政策共识制图（低中高重新排序）

In [5]:
import pandas as pd  
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
from pathlib import Path

class Config:
    FILE_PATH = "../data/4-3-Per-Policy_Intra-Cluster_Consensus_Analysis.csv"
    SUPP_TREND_FILE_PATH = "../data/3-3-Visualizing_Models_by_Country.csv"
    OUTPUT_DIR = "../data/4-3-Policy_Consensus_Map"

    SUBPLOT_WIDTH = 6
    SUBPLOT_HEIGHT = 8
    MAX_COLS = 3
    DPI = 300
    BACKGROUND_COLOR = "white"
    PLOT_BACKGROUND_COLOR = "none"

    BAR_HEIGHT = 0.75
    GROUP_SPACING = 1.3
    INTRA_GROUP_SPACING = 0.35
    BAR_ALPHA = 0.88
    BAR_EDGE_WIDTH = 1.2
    BAR_EDGE_COLOR = "#2c3e50"

    LABEL_OUTSIDE_MARGIN = 0.8

    HATCHES = ["", "///", "xxx"]          # 图例/样式按固定 a/b/c 顺序
    DEFAULT_CLUSTER_ORDER = ['a', 'b', 'c']
    CLUSTER_ID_MAP = {0: 'a', 1: 'b', 2: 'c'}

    L1_COLORS = {
        "default_1": "#3498db", "default_2": "#e74c3c", "default_3": "#2ecc71",
        "default_4": "#f39c12", "default_5": "#9b59b6", "default_6": "#1abc9c",
        "default_7": "#34495e",
    }

    FONT_MAIN = "Microsoft YaHei"
    FONT_SIZE_TITLE = 14
    FONT_SIZE_AXIS_LABEL = 12
    FONT_SIZE_POLICY_NAME = 10
    FONT_SIZE_FEATURE = 6.5
    FONT_SIZE_VALUE = 6.5

    X_AXIS_LIMIT = (0, 115)
    X_AXIS_TICKS = np.arange(0, 101, 20)


class DataProcessor:
    """读取与准备数据；提供排序、标签与筛选."""

    def __init__(self, file_path: str, supp_trend_file_path: str):
        self.df = pd.read_csv(file_path, sep=None, engine='python', encoding='utf-8-sig')
        self._preprocess_data()
        self.supp_df = pd.read_csv(supp_trend_file_path, sep=None, engine='python', encoding='utf-8-sig')
        self.supp_trend_map = self._create_supp_trend_mapping()
        self.rank_start = {"低位": 0, "中位": 1, "高位": 2}
        self.rank_end   = {"低位": 0, "中位": 1, "高位": 2}
        self.rank_trend = {"上升": 0, "平稳": 1, "波动": 2, "下降": 3}

    def _preprocess_data(self):
        if self.df["共识占比"].dtype == "object":
            self.df["共识占比_数值"] = self.df["共识占比"].astype(str).str.rstrip("%").astype(float)
            if self.df["共识占比_数值"].max() <= 1:
                self.df["共识占比_数值"] *= 100
        else:
            self.df["共识占比_数值"] = self.df["共识占比"].astype(float)
            if self.df["共识占比_数值"].max() <= 1:
                self.df["共识占比_数值"] *= 100

        self.df["K值"] = self.df["K值"].astype(int)
        self.df["共识聚类ID"] = self.df["共识聚类ID"].astype(int)

        def to_letter(v):
            try:
                iv = int(v)
                return Config.CLUSTER_ID_MAP.get(iv, str(v).lower())
            except Exception:
                return str(v).lower()
        self.df["政策聚类ID_字母"] = self.df["政策聚类ID"].apply(to_letter)

    def _create_supp_trend_mapping(self):
        """(中文名/代码, 聚类字母) -> 'Starting+Trend+Ending' 映射."""
        m = {}
        if self.supp_df is None or self.supp_df.empty:
            return m

        def norm_cluster(x):
            s = str(x).strip().lower()
            if s in ["0", "1", "2"]:
                return Config.CLUSTER_ID_MAP.get(int(s), s)
            return s

        for _, row in self.supp_df.iterrows():
            pol_cn = str(row.get("L2政策中文名", "")).strip()
            pol_code = str(row.get("L2政策", "")).strip()
            cluster = norm_cluster(row.get("聚类ID", ""))
            s = str(row.get("Starting", "")).strip()
            t = str(row.get("Trend", "")).strip()
            e = str(row.get("Ending", "")).strip()
            label = "+".join([x for x in [s, t, e] if x])
            if label:
                if pol_cn:
                    m[("cn", pol_cn, cluster)] = label
                if pol_code:
                    m[("code", pol_code, cluster)] = label
        return m

    def _supp_fields_for(self, policy_cn: str, cluster_letter: str):
        """取补充文件(Starting,Ending,Trend)，无则(None,None,None)."""
        def norm_series_cl(sr):
            s = sr.astype(str).str.strip().str.lower()
            return s.map(lambda x: Config.CLUSTER_ID_MAP.get(int(x), x) if x in ["0", "1", "2"] else x)

        sub = self.supp_df[
            (self.supp_df.get("L2政策中文名") == policy_cn) &
            (norm_series_cl(self.supp_df.get("聚类ID")) == cluster_letter)
        ]
        if not sub.empty:
            r = sub.iloc[0]
            return (str(r.get("Starting", "") or None),
                    str(r.get("Ending", "") or None),
                    str(r.get("Trend", "") or None))
        return None, None, None

    def get_trend_label(self, policy_cn: str, policy_code: str, cluster_letter: str) -> str:
        """优先主数据的三字段；缺失则回退补充映射."""
        row = self.df[(self.df["L2共识政策中文名"] == policy_cn) &
                      (self.df["政策聚类ID_字母"] == cluster_letter)]
        if not row.empty:
            s = str(row.iloc[0].get("Starting", "")).strip()
            t = str(row.iloc[0].get("Trend", "")).strip()
            e = str(row.iloc[0].get("Ending", "")).strip()
            if s or t or e:
                return "+".join([x for x in [s, t, e] if x])
        label = self.supp_trend_map.get(("cn", policy_cn, cluster_letter), "")
        if label:
            return label
        if policy_code:
            return self.supp_trend_map.get(("code", policy_code, cluster_letter), "")
        return ""

    def get_cluster_order_for_policy(self, policy_cn: str) -> list:
        """
        返回该政策内部三条(a/b/c)的从上到下顺序：
        先Starting(低<中<高) -> 再Ending(低<中<高) -> 再Trend(上升<平稳<波动<下降).
        """
        items = []
        for cl in Config.DEFAULT_CLUSTER_ORDER:
            s, e, t = self._supp_fields_for(policy_cn, cl)
            if s is None and e is None and t is None:
                row = self.df[(self.df["L2共识政策中文名"] == policy_cn) &
                              (self.df["政策聚类ID_字母"] == cl)]
                if not row.empty:
                    s = str(row.iloc[0].get("Starting", "") or None)
                    e = str(row.iloc[0].get("Ending", "") or None)
                    t = str(row.iloc[0].get("Trend", "") or None)
            rs = self.rank_start.get(s, 99)
            re = self.rank_end.get(e, 99)
            rt = self.rank_trend.get(t, 99)
            if rs != 99 or re != 99 or rt != 99:
                items.append((cl, rs, re, rt))

        items.sort(key=lambda x: (x[1], x[2], x[3], Config.DEFAULT_CLUSTER_ORDER.index(x[0])))
        ordered = [cl for cl, *_ in items]
        for cl in Config.DEFAULT_CLUSTER_ORDER:
            if cl not in ordered:
                ordered.append(cl)
        return ordered

    def get_unique_values(self, col_name: str) -> list:
        return sorted(self.df[col_name].unique())

    def filter_data(self, k_val: int, consensus_cluster_id: int) -> pd.DataFrame:
        return self.df[(self.df["K值"] == k_val) & (self.df["共识聚类ID"] == consensus_cluster_id)].copy()

    def get_all_policies_for_k_sorted_by_l1(self, k_val: int) -> list:
        k_data = self.df[self.df["K值"] == k_val]
        policy_l1_map = k_data.groupby("L2共识政策中文名")["L1分类"].first()
        return policy_l1_map.sort_values().index.tolist()


class PolicyConsensusPlotter:
    """绘制组合图；条形风格不变，仅按新顺序定位y坐标."""

    def __init__(self, config: Config, data_processor: DataProcessor):
        self.config = config
        self.dp = data_processor
        self._init_matplotlib()

    def _init_matplotlib(self):
        plt.rcParams.update({
            "font.sans-serif": [self.config.FONT_MAIN, "Arial Unicode MS"],
            "axes.unicode_minus": False,
            "figure.dpi": self.config.DPI,
            "savefig.dpi": self.config.DPI,
            "grid.linestyle": "--",
            "grid.alpha": 0.25,
            "grid.linewidth": 0.7,
            "grid.color": "#95a5a6"
        })

    def _calc_y_positions(self, all_policies: list):
        """计算y坐标：对每个政策使用"从上到下"的内部顺序."""
        y_positions, y_labels, cluster_mapping = [], [], {}
        current_y = 0
        for policy in all_policies:
            policy_positions = {}
            ordered_letters = self.dp.get_cluster_order_for_policy(policy)  # top→bottom
            for cluster_letter in ordered_letters:
                y_positions.append(current_y)
                y_labels.append("")
                policy_positions[cluster_letter] = current_y
                current_y += 1 + self.config.INTRA_GROUP_SPACING
            cluster_mapping[policy] = policy_positions
            current_y += self.config.GROUP_SPACING
        return y_positions, y_labels, cluster_mapping

    def _create_gradient_color(self, base_color: str, alpha: float = 0.88):
        from matplotlib.colors import to_rgba
        rgba = to_rgba(base_color)
        return (*rgba[:3], alpha)

    def _draw_value_label(self, ax: plt.Axes, x_end: float, y: float, label: str):
        ax.text(x_end + self.config.LABEL_OUTSIDE_MARGIN, y, label,
                va="center", ha="left",
                fontsize=self.config.FONT_SIZE_VALUE, fontweight="bold",
                color="#2c3e50", zorder=6)

    def _draw_policy_labels(self, ax: plt.Axes, data: pd.DataFrame, cluster_mapping: dict):
        for policy in cluster_mapping.keys():
            positions = cluster_mapping[policy]
            policy_data = data[data["L2共识政策中文名"] == policy]
            policy_code = "" if policy_data.empty else str(policy_data["L2共识政策"].iloc[0])
            y_center = np.mean(list(positions.values()))
            ax.text(-26.5, y_center, policy, va="center", ha="right",
                    fontsize=self.config.FONT_SIZE_POLICY_NAME, fontweight="bold",
                    color="#2c3e50", zorder=10)
            for cluster_letter, y_pos in positions.items():
                label = self.dp.get_trend_label(policy, policy_code, cluster_letter)
                ax.text(-2.5, y_pos, label, va="center", ha="right",
                        fontsize=self.config.FONT_SIZE_FEATURE, fontweight="bold",
                        color="#4a4a4a", zorder=10)

    def _draw_bars(self, ax: plt.Axes, data: pd.DataFrame, cluster_mapping: dict, l1_color_map: dict):
        for policy in cluster_mapping.keys():
            # 获取该政策的排序后的聚类顺序（用于hatch索引）
            ordered_letters = self.dp.get_cluster_order_for_policy(policy)
            # 获取当前子图中该政策的数据（可能为空）
            policy_data = data[data["L2共识政策中文名"] == policy]
            # 获取L1分类（无数据时用默认颜色）
            l1_type = policy_data["L1分类"].iloc[0] if not policy_data.empty else "default_1"
            base_color = l1_color_map.get(l1_type, "#95A5A6")

            # 遍历该政策的所有聚类（确保每个位置都有占位）
            for cluster_letter, y_pos in cluster_mapping[policy].items():
                # 检查当前聚类是否有数据
                cluster_data = policy_data[policy_data["政策聚类ID_字母"] == cluster_letter] if not policy_data.empty else None
                
                # 计算hatch索引（基于排序后的顺序）
                hatch_idx = ordered_letters.index(cluster_letter)
                
                if cluster_data is not None and not cluster_data.empty:
                    # 有数据：绘制正常条形图
                    consensus_val = float(cluster_data["共识占比_数值"].iloc[0])
                    bar_color = self._create_gradient_color(base_color, self.config.BAR_ALPHA)
                    ax.barh(y=y_pos, width=consensus_val, height=self.config.BAR_HEIGHT,
                            color=bar_color, edgecolor=self.config.BAR_EDGE_COLOR,
                            linewidth=self.config.BAR_EDGE_WIDTH, hatch=self.config.HATCHES[hatch_idx],
                            alpha=1.0, zorder=3)
                    self._draw_value_label(ax, consensus_val, y_pos, f"{consensus_val:.1f}%")
                else:
                    # 无数据：绘制透明空条形图占位（宽度0，不显示但占位置）
                    ax.barh(y=y_pos, width=0, height=self.config.BAR_HEIGHT,
                            color=(0, 0, 0, 0),  # 完全透明
                            edgecolor=(0, 0, 0, 0),  # 边缘透明
                            linewidth=self.config.BAR_EDGE_WIDTH, 
                            hatch=self.config.HATCHES[hatch_idx],  # 保持hatch一致性（虽然透明但占位逻辑统一）
                            alpha=1.0, zorder=3)
        
        self._draw_policy_labels(ax, data, cluster_mapping)

    def _set_axes_style(self, ax: plt.Axes, y_pos: list, y_labels: list, cluster_id: int, countries_count: int = None):
        """设置坐标轴及子标题；子标题追加簇内国家数。"""
        ax.set_facecolor(self.config.PLOT_BACKGROUND_COLOR)
        ax.set_xlim(self.config.X_AXIS_LIMIT)
        ax.set_xticks(self.config.X_AXIS_TICKS)
        ax.set_xlabel("共识占比 (%)", fontsize=self.config.FONT_SIZE_AXIS_LABEL,
                      fontweight="bold", labelpad=10, color="#2c3e50")
        ax.xaxis.grid(True, zorder=0)
        ax.yaxis.grid(False)
        ax.set_yticks(y_pos)
        ax.set_yticklabels(y_labels)
        ax.set_ylabel("政策名称", fontsize=self.config.FONT_SIZE_AXIS_LABEL,
                      fontweight="bold", labelpad=122, color="#2c3e50")
        ax.invert_yaxis()
        # 子标题增加（簇内国家数：N）
        subtitle = f"共识聚类ID = {cluster_id}"
        if countries_count is not None:
            subtitle += f"（簇内国家数：{countries_count}）"
        ax.set_title(subtitle, fontsize=self.config.FONT_SIZE_TITLE,
                     fontweight="bold", pad=15, color="#2c3e50")
        for spine in ax.spines.values():
            spine.set_edgecolor("#bdc3c7"); spine.set_linewidth(1.5)
        ax.spines["top"].set_visible(False)
        ax.spines["right"].set_visible(False)

    def _add_legend(self, fig: plt.Figure):
        elems = [mpatches.Patch(facecolor="white", edgecolor=self.config.BAR_EDGE_COLOR,
                                hatch=self.config.HATCHES[i], linewidth=1.5,
                                label=f"政策聚类 {Config.DEFAULT_CLUSTER_ORDER[i].upper()}")
                 for i in range(len(Config.DEFAULT_CLUSTER_ORDER))]
        fig.legend(handles=elems, loc="lower center", ncol=len(Config.DEFAULT_CLUSTER_ORDER),
                   fontsize=self.config.FONT_SIZE_POLICY_NAME - 1, frameon=True,
                   framealpha=0.95, edgecolor="#bdc3c7", bbox_to_anchor=(0.5, 0.01))

    def generate_combined_plot(self, k_val: int, cluster_data_list: list, all_policies: list, l1_color_map: dict):
        n_clusters = len(cluster_data_list)
        n_cols = min(n_clusters, self.config.MAX_COLS)
        n_rows = int(np.ceil(n_clusters / n_cols))
        fig, axes = plt.subplots(n_rows, n_cols,
                                 figsize=(self.config.SUBPLOT_WIDTH * n_cols,
                                          self.config.SUBPLOT_HEIGHT * n_rows + 1),
                                 facecolor=self.config.BACKGROUND_COLOR)
        if n_clusters == 1:
            axes = np.array([axes])
        axes = axes.flatten()
        y_positions, y_labels, cluster_mapping = self._calc_y_positions(all_policies)
        for idx, (cluster_id, data) in enumerate(cluster_data_list):
            ax = axes[idx]
            self._draw_bars(ax, data, cluster_mapping, l1_color_map)
            # 取该子图对应的簇内国家数（该K与聚类下相同，取首个即可）
            countries_count = None
            if "簇内国家数" in data.columns and not data["簇内国家数"].empty:
                countries_count = int(pd.to_numeric(data["簇内国家数"], errors="coerce").dropna().iloc[0])
            self._set_axes_style(ax, y_positions, y_labels, cluster_id, countries_count)
        for idx in range(n_clusters, len(axes)):
            axes[idx].axis('off')
        fig.suptitle(f"政策共识占比分析 (K值 = {k_val})",
                     fontsize=self.config.FONT_SIZE_TITLE + 4, fontweight="bold", y=0.98, color="#2c3e50")
        self._add_legend(fig)
        plt.tight_layout(rect=[0, 0.03, 1, 0.96])
        output_path = Path(self.config.OUTPUT_DIR) / f"K{k_val}-Consensus_Policy_Composite_Chart.png"
        plt.savefig(output_path, bbox_inches="tight",
                    facecolor=self.config.BACKGROUND_COLOR, edgecolor="none", dpi=self.config.DPI)
        plt.close(fig)
        return str(output_path)


def main():
    config = Config()
    dp = DataProcessor(config.FILE_PATH, config.SUPP_TREND_FILE_PATH)
    plotter = PolicyConsensusPlotter(config, dp)

    Path(config.OUTPUT_DIR).mkdir(parents=True, exist_ok=True)
    print(f"输出文件夹：{Path(config.OUTPUT_DIR).resolve()}")

    k_values = dp.get_unique_values("K值")
    l1_types = dp.get_unique_values("L1分类")
    l1_color_list = list(config.L1_COLORS.values())
    l1_color_map = {l1: l1_color_list[i % len(l1_color_list)] for i, l1 in enumerate(sorted(l1_types))}

    for k in k_values:
        all_policies = dp.get_all_policies_for_k_sorted_by_l1(k)
        consensus_clusters = sorted(dp.df[dp.df["K值"] == k]["共识聚类ID"].unique())
        cluster_data_list = [(cid, dp.filter_data(k, cid)) for cid in consensus_clusters]
        saved_path = plotter.generate_combined_plot(k, cluster_data_list, all_policies, l1_color_map)
        print(f"生成：{Path(saved_path).name}")


if __name__ == "__main__":
    main()

输出文件夹：F:\Desktop\CAMPF_Supplementary\data\4-3-Policy_Consensus_Map
生成：K2-Consensus_Policy_Composite_Chart.png
生成：K3-Consensus_Policy_Composite_Chart.png
生成：K4-Consensus_Policy_Composite_Chart.png
生成：K5-Consensus_Policy_Composite_Chart.png
生成：K6-Consensus_Policy_Composite_Chart.png
生成：K7-Consensus_Policy_Composite_Chart.png
生成：K8-Consensus_Policy_Composite_Chart.png
生成：K9-Consensus_Policy_Composite_Chart.png
生成：K10-Consensus_Policy_Composite_Chart.png
生成：K11-Consensus_Policy_Composite_Chart.png
生成：K12-Consensus_Policy_Composite_Chart.png
生成：K13-Consensus_Policy_Composite_Chart.png
生成：K14-Consensus_Policy_Composite_Chart.png
生成：K15-Consensus_Policy_Composite_Chart.png
生成：K16-Consensus_Policy_Composite_Chart.png
生成：K17-Consensus_Policy_Composite_Chart.png
生成：K18-Consensus_Policy_Composite_Chart.png
生成：K19-Consensus_Policy_Composite_Chart.png
生成：K20-Consensus_Policy_Composite_Chart.png
生成：K21-Consensus_Policy_Composite_Chart.png


### 筛选占比大于等于50的

In [ ]:
import pandas as pd  
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
from pathlib import Path

class Config:
    FILE_PATH = "../data/4-3-Per-Policy_Intra-Cluster_Consensus_Analysis.csv"
    SUPP_TREND_FILE_PATH = "../data/3-3-Visualizing_Models_by_Country.csv"
    OUTPUT_DIR = "../data/4-3-Policy_Consensus_Map/Consensus_threshold_greater_than_50"
    FILTERED_OUTPUT_PATH = "../data/4-3-Policy_Consensus_Map/Consensus_threshold_greater_than_50/4-3-Filtered_High_Consensus_Policies.csv"

    SUBPLOT_WIDTH = 6
    SUBPLOT_HEIGHT = 8
    MAX_COLS = 3
    DPI = 300
    BACKGROUND_COLOR = "white"
    PLOT_BACKGROUND_COLOR = "none"

    BAR_HEIGHT = 0.75
    GROUP_SPACING = 1.3
    INTRA_GROUP_SPACING = 0.35
    BAR_ALPHA = 0.88
    BAR_EDGE_WIDTH = 1.2
    BAR_EDGE_COLOR = "#2c3e50"

    LABEL_OUTSIDE_MARGIN = 0.8

    HATCHES = ["", "///", "xxx"]
    DEFAULT_CLUSTER_ORDER = ['a', 'b', 'c']
    CLUSTER_ID_MAP = {0: 'a', 1: 'b', 2: 'c'}

    L1_COLORS = {
        "default_1": "#3498db", "default_2": "#e74c3c", "default_3": "#2ecc71",
        "default_4": "#f39c12", "default_5": "#9b59b6", "default_6": "#1abc9c",
        "default_7": "#34495e",
    }

    FONT_MAIN = "Microsoft YaHei"
    FONT_SIZE_TITLE = 14
    FONT_SIZE_AXIS_LABEL = 12
    FONT_SIZE_POLICY_NAME = 10
    FONT_SIZE_FEATURE = 6.5
    FONT_SIZE_VALUE = 6.5

    X_AXIS_LIMIT = (0, 115)
    X_AXIS_TICKS = np.arange(0, 101, 20)

    CONSENSUS_THRESHOLD = 50.0
    HIGH_CONSENSUS_COLOR = "#e74c3c"
    NORMAL_FEATURE_COLOR = "#4a4a4a"
    HIGH_CONSENSUS_POLICY_COLOR = "#ff8c00"
    NORMAL_POLICY_COLOR = "#2c3e50"


class DataProcessor:
    def __init__(self, file_path: str, supp_trend_file_path: str):
        self.df = pd.read_csv(file_path, sep=None, engine='python', encoding='utf-8-sig')
        self._preprocess_data()
        self.supp_df = pd.read_csv(supp_trend_file_path, sep=None, engine='python', encoding='utf-8-sig')
        self.supp_trend_map = self._create_supp_trend_mapping()
        self.rank_start = {"低位": 0, "中位": 1, "高位": 2}
        self.rank_end = {"低位": 0, "中位": 1, "高位": 2}
        self.rank_trend = {"上升": 0, "平稳": 1, "波动": 2, "下降": 3}

    def _preprocess_data(self):
        if self.df["共识占比"].dtype == "object":
            self.df["共识占比_数值"] = self.df["共识占比"].astype(str).str.rstrip("%").astype(float)
            if self.df["共识占比_数值"].max() <= 1:
                self.df["共识占比_数值"] *= 100
        else:
            self.df["共识占比_数值"] = self.df["共识占比"].astype(float)
            if self.df["共识占比_数值"].max() <= 1:
                self.df["共识占比_数值"] *= 100

        self.df["K值"] = self.df["K值"].astype(int)
        self.df["共识聚类ID"] = self.df["共识聚类ID"].astype(int)

        def to_letter(v):
            try:
                iv = int(v)
                return Config.CLUSTER_ID_MAP.get(iv, str(v).lower())
            except Exception:
                return str(v).lower()
        self.df["政策聚类ID_字母"] = self.df["政策聚类ID"].apply(to_letter)

    def _create_supp_trend_mapping(self):
        m = {}
        if self.supp_df is None or self.supp_df.empty:
            return m

        def norm_cluster(x):
            s = str(x).strip().lower()
            if s in ["0", "1", "2"]:
                return Config.CLUSTER_ID_MAP.get(int(s), s)
            return s

        for _, row in self.supp_df.iterrows():
            pol_cn = str(row.get("L2政策中文名", "")).strip()
            pol_code = str(row.get("L2政策", "")).strip()
            cluster = norm_cluster(row.get("聚类ID", ""))
            s = str(row.get("Starting", "")).strip()
            t = str(row.get("Trend", "")).strip()
            e = str(row.get("Ending", "")).strip()
            label = "+".join([x for x in [s, t, e] if x])
            if label:
                if pol_cn:
                    m[("cn", pol_cn, cluster)] = label
                if pol_code:
                    m[("code", pol_code, cluster)] = label
        return m

    def _supp_fields_for(self, policy_cn: str, cluster_letter: str):
        def norm_series_cl(sr):
            s = sr.astype(str).str.strip().str.lower()
            return s.map(lambda x: Config.CLUSTER_ID_MAP.get(int(x), x) if x in ["0", "1", "2"] else x)

        sub = self.supp_df[
            (self.supp_df.get("L2政策中文名") == policy_cn) &
            (norm_series_cl(self.supp_df.get("聚类ID")) == cluster_letter)
        ]
        if not sub.empty:
            r = sub.iloc[0]
            return (str(r.get("Starting", "") or None),
                    str(r.get("Ending", "") or None),
                    str(r.get("Trend", "") or None))
        return None, None, None

    def get_trend_label(self, policy_cn: str, policy_code: str, cluster_letter: str) -> str:
        row = self.df[(self.df["L2共识政策中文名"] == policy_cn) &
                      (self.df["政策聚类ID_字母"] == cluster_letter)]
        if not row.empty:
            s = str(row.iloc[0].get("Starting", "")).strip()
            t = str(row.iloc[0].get("Trend", "")).strip()
            e = str(row.iloc[0].get("Ending", "")).strip()
            if s or t or e:
                return "+".join([x for x in [s, t, e] if x])
        label = self.supp_trend_map.get(("cn", policy_cn, cluster_letter), "")
        if label:
            return label
        if policy_code:
            return self.supp_trend_map.get(("code", policy_code, cluster_letter), "")
        return ""

    def get_cluster_order_for_policy(self, policy_cn: str) -> list:
        """
        返回该政策内部三条(a/b/c)的从上到下顺序：
        先Starting(低<中<高) -> 再Ending(低<中<高) -> 再Trend(上升<平稳<波动<下降).
        """
        items = []
        for cl in Config.DEFAULT_CLUSTER_ORDER:
            s, e, t = self._supp_fields_for(policy_cn, cl)
            if s is None and e is None and t is None:
                row = self.df[(self.df["L2共识政策中文名"] == policy_cn) &
                              (self.df["政策聚类ID_字母"] == cl)]
                if not row.empty:
                    s = str(row.iloc[0].get("Starting", "") or None)
                    e = str(row.iloc[0].get("Ending", "") or None)
                    t = str(row.iloc[0].get("Trend", "") or None)
            rs = self.rank_start.get(s, 99)
            re = self.rank_end.get(e, 99)
            rt = self.rank_trend.get(t, 99)
            if rs != 99 or re != 99 or rt != 99:
                items.append((cl, rs, re, rt))

        items.sort(key=lambda x: (x[1], x[2], x[3], Config.DEFAULT_CLUSTER_ORDER.index(x[0])))
        ordered = [cl for cl, *_ in items]
        for cl in Config.DEFAULT_CLUSTER_ORDER:
            if cl not in ordered:
                ordered.append(cl)
        return ordered

    def get_unique_values(self, col_name: str) -> list:
        return sorted(self.df[col_name].unique())

    def filter_data(self, k_val: int, consensus_cluster_id: int) -> pd.DataFrame:
        return self.df[(self.df["K值"] == k_val) & (self.df["共识聚类ID"] == consensus_cluster_id)].copy()

    def get_all_policies_for_k_sorted_by_l1(self, k_val: int) -> list:
        k_data = self.df[self.df["K值"] == k_val]
        policy_l1_map = k_data.groupby("L2共识政策中文名")["L1分类"].first()
        return policy_l1_map.sort_values().index.tolist()

    def is_high_consensus(self, k_val: int, consensus_cluster_id: int, policy_cn: str,
                          cluster_letter: str, threshold: float = 50.0) -> bool:
        row = self.df[
            (self.df["K值"] == k_val) &
            (self.df["共识聚类ID"] == consensus_cluster_id) &
            (self.df["L2共识政策中文名"] == policy_cn) &
            (self.df["政策聚类ID_字母"] == cluster_letter)
        ]
        if not row.empty:
            consensus_val = float(row.iloc[0]["共识占比_数值"])
            return consensus_val >= threshold
        return False

    def has_high_consensus_cluster(self, k_val: int, consensus_cluster_id: int,
                                   policy_cn: str, threshold: float = 50.0) -> bool:
        """判断该政策是否有任意一个聚类达到高共识度阈值"""
        for cluster_letter in Config.DEFAULT_CLUSTER_ORDER:
            if self.is_high_consensus(k_val, consensus_cluster_id, policy_cn, cluster_letter, threshold):
                return True
        return False

    def filter_high_consensus_policies(self, threshold: float = 50.0) -> pd.DataFrame:
        """筛选共识度大于等于threshold%的所有记录"""
        filtered = self.df[self.df["共识占比_数值"] >= threshold].copy()
        
        output_data = []
        
        for _, row in filtered.iterrows():
            k_val = int(row["K值"])
            cluster_id = int(row["共识聚类ID"])
            
            same_group = filtered[
                (filtered["K值"] == k_val) & 
                (filtered["共识聚类ID"] == cluster_id)
            ]
            筛选政策数 = len(same_group)
            
            output_data.append({
                "K值": k_val,
                "共识聚类ID": cluster_id,
                "簇内国家数": row.get("簇内国家数", None),
                "筛选政策数": 筛选政策数,
                "L2共识政策": row["L2共识政策"],
                "L2共识政策中文名": row["L2共识政策中文名"],
                "L1分类": row["L1分类"],
                "L1分类中文名": row["L1分类中文名"],
                "政策聚类ID": row["政策聚类ID_字母"],
                "Starting": row.get("Starting", ""),
                "Trend": row.get("Trend", ""),
                "Ending": row.get("Ending", ""),
                "政策国家数": row["政策国家数"],
                "共识占比": f"{row['共识占比_数值']:.2f}%"
            })
        
        result_df = pd.DataFrame(output_data)
        
        if not result_df.empty:
            result_df = result_df.sort_values(
                by=["K值", "共识聚类ID", "L2共识政策中文名", "政策聚类ID"],
                ascending=[True, True, True, True]
            )
        
        return result_df


class PolicyConsensusPlotter:
    """绘制组合图；条形风格不变，仅按新顺序定位y坐标."""

    def __init__(self, config: Config, data_processor: DataProcessor):
        self.config = config
        self.dp = data_processor
        self._init_matplotlib()

    def _init_matplotlib(self):
        plt.rcParams.update({
            "font.sans-serif": [self.config.FONT_MAIN, "Arial Unicode MS"],
            "axes.unicode_minus": False,
            "figure.dpi": self.config.DPI,
            "savefig.dpi": self.config.DPI,
            "grid.linestyle": "--",
            "grid.alpha": 0.25,
            "grid.linewidth": 0.7,
            "grid.color": "#95a5a6"
        })

    def _calc_y_positions(self, all_policies: list):
        """计算y坐标：对每个政策使用"从上到下"的内部顺序."""
        y_positions, y_labels, cluster_mapping = [], [], {}
        current_y = 0
        for policy in all_policies:
            policy_positions = {}
            ordered_letters = self.dp.get_cluster_order_for_policy(policy)  # top→bottom
            for cluster_letter in ordered_letters:
                y_positions.append(current_y)
                y_labels.append("")
                policy_positions[cluster_letter] = current_y
                current_y += 1 + self.config.INTRA_GROUP_SPACING
            cluster_mapping[policy] = policy_positions
            current_y += self.config.GROUP_SPACING
        return y_positions, y_labels, cluster_mapping

    def _create_gradient_color(self, base_color: str, alpha: float = 0.88):
        from matplotlib.colors import to_rgba
        rgba = to_rgba(base_color)
        return (*rgba[:3], alpha)

    def _draw_value_label(self, ax: plt.Axes, x_end: float, y: float, label: str):
        ax.text(x_end + self.config.LABEL_OUTSIDE_MARGIN, y, label,
                va="center", ha="left",
                fontsize=self.config.FONT_SIZE_VALUE, fontweight="bold",
                color="#2c3e50", zorder=6)

    def _draw_policy_labels(self, ax: plt.Axes, k_val: int, consensus_cluster_id: int,
                           data: pd.DataFrame, cluster_mapping: dict):
        for policy in cluster_mapping.keys():
            positions = cluster_mapping[policy]
            policy_data = data[data["L2共识政策中文名"] == policy]
            policy_code = "" if policy_data.empty else str(policy_data["L2共识政策"].iloc[0])
            y_center = np.mean(list(positions.values()))
            
            # 判断该政策是否有高共识度聚类
            has_high = self.dp.has_high_consensus_cluster(k_val, consensus_cluster_id, policy, self.config.CONSENSUS_THRESHOLD)
            policy_name_color = (self.config.HIGH_CONSENSUS_POLICY_COLOR if has_high
                                 else self.config.NORMAL_POLICY_COLOR)
            
            ax.text(-26.5, y_center, policy, va="center", ha="right",
                    fontsize=self.config.FONT_SIZE_POLICY_NAME, fontweight="bold",
                    color=policy_name_color, zorder=10)
            
            for cluster_letter, y_pos in positions.items():
                label = self.dp.get_trend_label(policy, policy_code, cluster_letter)
                
                # 判断该聚类是否为高共识度
                is_high = self.dp.is_high_consensus(k_val, consensus_cluster_id, policy, cluster_letter, self.config.CONSENSUS_THRESHOLD)
                label_color = (self.config.HIGH_CONSENSUS_COLOR if is_high
                               else self.config.NORMAL_FEATURE_COLOR)
                
                ax.text(-2.5, y_pos, label, va="center", ha="right",
                        fontsize=self.config.FONT_SIZE_FEATURE, fontweight="bold",
                        color=label_color, zorder=10)

    def _draw_bars(self, ax: plt.Axes, k_val: int, consensus_cluster_id: int,
                   data: pd.DataFrame, cluster_mapping: dict, l1_color_map: dict):
        for policy in cluster_mapping.keys():
            # 获取该政策的排序后的聚类顺序（用于hatch索引）
            ordered_letters = self.dp.get_cluster_order_for_policy(policy)
            # 获取当前子图中该政策的数据（可能为空）
            policy_data = data[data["L2共识政策中文名"] == policy]
            # 获取L1分类（无数据时用默认颜色）
            l1_type = policy_data["L1分类"].iloc[0] if not policy_data.empty else "default_1"
            base_color = l1_color_map.get(l1_type, "#95A5A6")

            # 遍历该政策的所有聚类（确保每个位置都有占位）
            for cluster_letter, y_pos in cluster_mapping[policy].items():
                # 检查当前聚类是否有数据
                cluster_data = policy_data[policy_data["政策聚类ID_字母"] == cluster_letter] if not policy_data.empty else None
                
                # 计算hatch索引（基于排序后的顺序）
                hatch_idx = ordered_letters.index(cluster_letter)
                
                if cluster_data is not None and not cluster_data.empty:
                    # 有数据：绘制正常条形图
                    consensus_val = float(cluster_data["共识占比_数值"].iloc[0])
                    bar_color = self._create_gradient_color(base_color, self.config.BAR_ALPHA)
                    ax.barh(y=y_pos, width=consensus_val, height=self.config.BAR_HEIGHT,
                            color=bar_color, edgecolor=self.config.BAR_EDGE_COLOR,
                            linewidth=self.config.BAR_EDGE_WIDTH, hatch=self.config.HATCHES[hatch_idx],
                            alpha=1.0, zorder=3)
                    self._draw_value_label(ax, consensus_val, y_pos, f"{consensus_val:.1f}%")
                else:
                    # 无数据：绘制透明空条形图占位（宽度0，不显示但占位置）
                    ax.barh(y=y_pos, width=0, height=self.config.BAR_HEIGHT,
                            color=(0, 0, 0, 0),  # 完全透明
                            edgecolor=(0, 0, 0, 0),  # 边缘透明
                            linewidth=self.config.BAR_EDGE_WIDTH, 
                            hatch=self.config.HATCHES[hatch_idx],  # 保持hatch一致性
                            alpha=1.0, zorder=3)
        
        self._draw_policy_labels(ax, k_val, consensus_cluster_id, data, cluster_mapping)

    def _set_axes_style(self, ax: plt.Axes, y_pos: list, y_labels: list, cluster_id: int, countries_count: int = None):
        """设置坐标轴及子标题；子标题追加簇内国家数。"""
        ax.set_facecolor(self.config.PLOT_BACKGROUND_COLOR)
        ax.set_xlim(self.config.X_AXIS_LIMIT)
        ax.set_xticks(self.config.X_AXIS_TICKS)
        ax.set_xlabel("共识占比 (%)", fontsize=self.config.FONT_SIZE_AXIS_LABEL,
                      fontweight="bold", labelpad=10, color="#2c3e50")
        ax.xaxis.grid(True, zorder=0)
        ax.yaxis.grid(False)
        ax.set_yticks(y_pos)
        ax.set_yticklabels(y_labels)
        ax.set_ylabel("政策名称", fontsize=self.config.FONT_SIZE_AXIS_LABEL,
                      fontweight="bold", labelpad=122, color="#2c3e50")
        ax.invert_yaxis()
        # 子标题增加（簇内国家数：N）
        subtitle = f"共识聚类ID = {cluster_id}"
        if countries_count is not None:
            subtitle += f"（簇内国家数：{countries_count}）"
        ax.set_title(subtitle, fontsize=self.config.FONT_SIZE_TITLE,
                     fontweight="bold", pad=15, color="#2c3e50")
        for spine in ax.spines.values():
            spine.set_edgecolor("#bdc3c7"); spine.set_linewidth(1.5)
        ax.spines["top"].set_visible(False)
        ax.spines["right"].set_visible(False)

    def _add_legend(self, fig: plt.Figure):
        elems = [mpatches.Patch(facecolor="white", edgecolor=self.config.BAR_EDGE_COLOR,
                                hatch=self.config.HATCHES[i], linewidth=1.5,
                                label=f"政策聚类 {Config.DEFAULT_CLUSTER_ORDER[i].upper()}")
                 for i in range(len(Config.DEFAULT_CLUSTER_ORDER))]
        fig.legend(handles=elems, loc="lower center", ncol=len(Config.DEFAULT_CLUSTER_ORDER),
                   fontsize=self.config.FONT_SIZE_POLICY_NAME - 1, frameon=True,
                   framealpha=0.95, edgecolor="#bdc3c7", bbox_to_anchor=(0.5, 0.01))

    def generate_combined_plot(self, k_val: int, cluster_data_list: list, all_policies: list, l1_color_map: dict):
        n_clusters = len(cluster_data_list)
        n_cols = min(n_clusters, self.config.MAX_COLS)
        n_rows = int(np.ceil(n_clusters / n_cols))
        fig, axes = plt.subplots(n_rows, n_cols,
                                 figsize=(self.config.SUBPLOT_WIDTH * n_cols,
                                          self.config.SUBPLOT_HEIGHT * n_rows + 1),
                                 facecolor=self.config.BACKGROUND_COLOR)
        if n_clusters == 1:
            axes = np.array([axes])
        axes = axes.flatten()
        y_positions, y_labels, cluster_mapping = self._calc_y_positions(all_policies)
        for idx, (cluster_id, data) in enumerate(cluster_data_list):
            ax = axes[idx]
            self._draw_bars(ax, k_val, cluster_id, data, cluster_mapping, l1_color_map)
            # 取该子图对应的簇内国家数
            countries_count = None
            if "簇内国家数" in data.columns and not data["簇内国家数"].empty:
                countries_count = int(pd.to_numeric(data["簇内国家数"], errors="coerce").dropna().iloc[0])
            self._set_axes_style(ax, y_positions, y_labels, cluster_id, countries_count)
        for idx in range(n_clusters, len(axes)):
            axes[idx].axis('off')
        fig.suptitle(f"政策共识占比分析 (K值 = {k_val})",
                     fontsize=self.config.FONT_SIZE_TITLE + 4, fontweight="bold", y=0.98, color="#2c3e50")
        self._add_legend(fig)
        plt.tight_layout(rect=[0, 0.03, 1, 0.96])
        output_path = Path(self.config.OUTPUT_DIR) / f"K{k_val}-Consensus_Policy_Composite_Chart.png"
        plt.savefig(output_path, bbox_inches="tight",
                    facecolor=self.config.BACKGROUND_COLOR, edgecolor="none", dpi=self.config.DPI)
        plt.close(fig)
        return str(output_path)


def main():
    config = Config()
    dp = DataProcessor(config.FILE_PATH, config.SUPP_TREND_FILE_PATH)
    plotter = PolicyConsensusPlotter(config, dp)

    Path(config.OUTPUT_DIR).mkdir(parents=True, exist_ok=True)
    print(f"输出文件夹：{Path(config.OUTPUT_DIR).resolve()}")

    # 筛选并保存高共识度政策
    filtered_df = dp.filter_high_consensus_policies(config.CONSENSUS_THRESHOLD)
    if not filtered_df.empty:
        output_dir = Path(config.FILTERED_OUTPUT_PATH).parent
        output_dir.mkdir(parents=True, exist_ok=True)
        filtered_df.to_csv(config.FILTERED_OUTPUT_PATH, index=False, encoding='utf-8-sig')
        print(f"筛选结果CSV: {Path(config.FILTERED_OUTPUT_PATH).resolve()}")
        print(f"共筛选出 {len(filtered_df)} 条共识度>={config.CONSENSUS_THRESHOLD}%的记录")

    k_values = dp.get_unique_values("K值")
    l1_types = dp.get_unique_values("L1分类")
    l1_color_list = list(config.L1_COLORS.values())
    l1_color_map = {l1: l1_color_list[i % len(l1_color_list)] for i, l1 in enumerate(sorted(l1_types))}

    for k in k_values:
        all_policies = dp.get_all_policies_for_k_sorted_by_l1(k)
        consensus_clusters = sorted(dp.df[dp.df["K值"] == k]["共识聚类ID"].unique())
        cluster_data_list = [(cid, dp.filter_data(k, cid)) for cid in consensus_clusters]
        saved_path = plotter.generate_combined_plot(k, cluster_data_list, all_policies, l1_color_map)
        print(f"生成：{Path(saved_path).name}")


if __name__ == "__main__":
    main()

输出文件夹：F:\Desktop\CAMPF_Supplementary\data\4-3-Policy_Consensus_Map\Consensus_threshold_greater_than_50
筛选结果CSV: F:\Desktop\CAMPF_Supplementary\data\4-3-Policy_Consensus_Map\Consensus_threshold_greater_than_50\4-3-Filtered_High_Consensus_Policies.csv
共筛选出 961 条共识度>=50.0%的记录
生成：K2-Consensus_Policy_Composite_Chart.png
生成：K3-Consensus_Policy_Composite_Chart.png
生成：K4-Consensus_Policy_Composite_Chart.png
生成：K5-Consensus_Policy_Composite_Chart.png
生成：K6-Consensus_Policy_Composite_Chart.png
生成：K7-Consensus_Policy_Composite_Chart.png
生成：K8-Consensus_Policy_Composite_Chart.png
生成：K9-Consensus_Policy_Composite_Chart.png
生成：K10-Consensus_Policy_Composite_Chart.png
生成：K11-Consensus_Policy_Composite_Chart.png


### 筛选占比大于等于80的

In [3]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import pandas as pd
import json
import os
from general_pic_setup import setup_mpl_single2

setup_mpl_single2()

# 读取数据
df = pd.read_csv(r'F:\Desktop\CAMPF_Supplementary\data\4-3-K7-Per-Policy_Intra-Cluster_Consensus_Analysis.csv')
with open(r'F:\Desktop\CAMPF_Supplementary\data\3-2-Human_Recognition_Mode_txt.json', 'r', encoding='utf-8') as f:
    recognition_data = json.load(f)

output_dir = r'F:\Desktop\CAMPF_Supplementary\data\4-3-Policy_Consensus_Map\Consensus_threshold_greater_than_80\output'
os.makedirs(output_dir, exist_ok=True)


def get_combination_from_json(l1_category, l2_policy, cluster_id):
    """从JSON获取组合代码缩写，格式: L-R-M"""
    policy_data = recognition_data[l1_category][l2_policy][str(cluster_id)]
    starting_abbrev = policy_data['Starting'][0].upper()
    trend_abbrev = policy_data['Trend'][0].upper()
    ending_abbrev = policy_data['Ending'].split()[0][0].upper()
    return f"{starting_abbrev}-{trend_abbrev}-{ending_abbrev}"


def get_combination_details(l1_category, l2_policy, cluster_id):
    """获取完整组合信息用于排序"""
    policy_data = recognition_data[l1_category][l2_policy][str(cluster_id)]
    return {
        'starting': policy_data['Starting'][0].upper(),
        'ending': policy_data['Ending'].split()[0][0].upper(),
        'trend': policy_data['Trend'][0].upper(),
        'abbrev': f"{policy_data['Starting'][0].upper()}-{policy_data['Trend'][0].upper()}-{policy_data['Ending'].split()[0][0].upper()}"
    }


def get_sort_key(combo_details):
    """排序规则: Starting(L<M<H) > Ending(L<M<H) > Trend(R<S<F<D)"""
    starting_order = {'L': 0, 'M': 1, 'H': 2}
    ending_order = {'L': 0, 'M': 1, 'H': 2}
    trend_order = {'R': 0, 'S': 1, 'F': 2, 'D': 3}
    
    return (
        starting_order.get(combo_details['starting'], 999),
        ending_order.get(combo_details['ending'], 999),
        trend_order.get(combo_details['trend'], 999)
    )


# 颜色配置
color_groups = {
    'Incentive-based': {
        'base': '#FCF3DD',
        'colors': ['#D19D00', '#EBC04F', '#F8E5B8']
    },
    'Regulatory': {
        'base': '#EDF6E5',
        'colors': ['#5C9419', '#95BC65', '#C8E3B3']
    },
    'Commitment-based': {
        'base': '#EFEEF7',
        'colors': ['#6A63A3', '#9C98BE', '#CFCEDF']
    },
    'Research and Development (R&D)': {
        'base': '#FDEBF3',
        'colors': ['#D32679', '#ED6CA5', '#F9BDD9']
    }
}


# 直接获取所有共识聚类ID
consensus_cluster_ids = df['共识聚类ID'].unique()

for consensus_cluster_id in consensus_cluster_ids:
    print(f"\n正在生成共识聚类ID = {consensus_cluster_id} 的图表...")
    
    df_cluster = df[df['共识聚类ID'] == consensus_cluster_id].copy()
    
    # 添加组合列
    df_cluster['combination_json'] = df_cluster.apply(
        lambda row: get_combination_from_json(row['L1分类'], row['L2共识政策中文名'], row['政策聚类ID']),
        axis=1
    )
    
    l1_categories = df_cluster['L1分类'].unique()
    
    fig, ax = plt.subplots(figsize=(14, 11), facecolor='white')
    ax.set_facecolor('white')
    
    # 组织政策数据
    policy_groups = []
    policy_data = {}
    
    for l1_category in l1_categories:
        group_data = []
        df_l1 = df_cluster[df_cluster['L1分类'] == l1_category]
        
        for policy_name in df_l1['Abbrev_L2'].unique():
            df_policy = df_l1[df_l1['Abbrev_L2'] == policy_name].copy()
            l2_full_name = df_policy.iloc[0]['L2共识政策中文名']
            
            cluster_data_list = []
            for cluster_id in range(3):
                combo_details = get_combination_details(l1_category, l2_full_name, cluster_id)
                cluster_data = df_policy[df_policy['政策聚类ID'] == cluster_id]
                value = cluster_data.iloc[0]['共识占比'] * 100 if not cluster_data.empty else 0
                
                cluster_data_list.append({
                    'combo_details': combo_details,
                    'value': value,
                    'cluster_id': cluster_id
                })
            
            cluster_data_list.sort(key=lambda x: get_sort_key(x['combo_details']))
            
            policy_data[policy_name] = {
                'combinations': [item['combo_details']['abbrev'] for item in cluster_data_list],
                'values': [item['value'] for item in cluster_data_list],
                'cluster_ids': [item['cluster_id'] for item in cluster_data_list]
            }
            
            group_key = l1_category if l1_category in color_groups else 'Research and Development (R&D)'
            group_data.append((policy_name, group_key))
        
        if group_data:
            policy_groups.append(group_data)
    
    all_policies = [policy for group in policy_groups for policy in group]
    
    # 布局参数
    policy_center_x = -5.3
    combo_col_center = -2.2
    big_bar_height = 0.7
    small_bar_height = 0.18
    within_group_gap = small_bar_height * 0.3
    white_rect_height = small_bar_height * 1.2
    between_group_gap = white_rect_height
    
    # 计算Y位置
    y_positions = []
    group_boundaries = []
    current_y = 0
    for group_idx, group in enumerate(policy_groups):
        for policy_idx in range(len(group)):
            y_positions.append(current_y)
            if policy_idx < len(group) - 1:
                current_y += big_bar_height + within_group_gap
            else:
                current_y += big_bar_height
        
        if group_idx < len(policy_groups) - 1:
            gap_start = current_y
            current_y += between_group_gap
            gap_end = current_y
            group_boundaries.append((gap_start, gap_end))
    
    policy_label_color = '#000000'
    
    # 绘制每个政策条
    for y_pos, (policy_name, group) in zip(y_positions, all_policies):
        group_colors = color_groups[group]
        data = policy_data[policy_name]
        values = data['values']
        combinations = data['combinations']
        
        # 背景大bar
        ax.barh(y_pos, 100, height=big_bar_height, color=group_colors['base'], alpha=0.8)
        
        bar_top = y_pos + big_bar_height / 2
        bar_bottom = y_pos - big_bar_height / 2
        available_space = big_bar_height - 3 * small_bar_height
        gap = available_space / 4 - 0.01
        colors = group_colors['colors']
        
        # 三个小bar
        bar_positions = [
            bar_top - gap - small_bar_height / 2,
            y_pos,
            bar_bottom + gap + small_bar_height / 2
        ]
        
        for i, (bar_y, value, combo) in enumerate(zip(bar_positions, values, combinations)):
            bar_width = min(value, 100)
            ax.barh(bar_y, bar_width, height=small_bar_height, color=colors[i], alpha=0.95, zorder=3)
            
            # 数值标签
            if value > 0:
                text_x = min(bar_width - 0.8, 99.2)
                text_color = 'white' if i == 0 else 'black'
                ax.text(text_x, bar_y, f'{value:.1f}', va='center', ha='right',
                        fontsize=7, color=text_color, family='Arial')
            else:
                ax.text(1, bar_y, '0', va='center', ha='left',
                        fontsize=7, color='black', family='Arial')
            
            # 组合标签
            ax.text(combo_col_center, bar_y, combo, va='center', ha='center',
                    fontsize=8, color=policy_label_color, family='Arial')
        
        # 政策名称
        ax.text(policy_center_x, y_pos, policy_name, va='center', ha='center',
                fontsize=9, color=policy_label_color, family='Arial')
        
        # 左侧灰背景矩形框
        rect = mpatches.Rectangle(
            (policy_center_x - 1.5, y_pos - big_bar_height / 2 + 0.02),
            combo_col_center + 1.5 - (policy_center_x - 1.5),
            big_bar_height - 0.04,
            facecolor='#EBEBEB', edgecolor='#999999', linewidth=0.25
        )
        ax.add_patch(rect)
    
    # Y轴范围
    y_line_top = y_positions[0] - big_bar_height / 2
    y_line_bottom = y_positions[-1] + big_bar_height / 2
    
    ax.set_xlim(-6.8, 103)
    ax.set_ylim(y_line_bottom, y_line_top)
    
    # 50%和80%截断线
    ax.plot([50, 50], [y_line_top, y_line_bottom], color='#3a3a3a', 
            linewidth=2.8, linestyle='--', zorder=1000, alpha=0.9)
    ax.plot([80, 80], [y_line_top, y_line_bottom], color='#1f1f1f', 
            linewidth=2.8, linestyle='-', zorder=1000, alpha=0.9)
    
    # X轴设置
    ax.xaxis.tick_top()
    ax.xaxis.set_label_position('top')
    ax.set_xlabel('Percentage (%)', labelpad=15, color='#2c3e50')
    ax.set_xticks([0, 20, 40, 60, 80, 100])
    ax.tick_params(axis='x', colors='#2c3e50', length=6, width=1.5)
    
    ax.set_yticks([])
    ax.yaxis.set_label_coords(-0.02, 0.02)
    
    # 左侧主线
    ax.plot([0, 0], [y_line_top, y_line_bottom], color='#34495e', 
            linewidth=2, zorder=10, clip_on=False)
    
    # 边框
    ax.spines['top'].set_visible(True)
    ax.spines['top'].set_color('#34495e')
    ax.spines['top'].set_linewidth(2)
    ax.spines['top'].set_bounds(0, 100)
    ax.spines['top'].set_zorder(10)
    ax.spines['bottom'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
    
    # 每个L1组左侧刻度
    tick_length = 0.3
    current_idx = 0
    for group in policy_groups:
        group_y_positions = y_positions[current_idx:current_idx + len(group)]
        
        if group_y_positions:
            y_start = group_y_positions[0] + big_bar_height / 2
            y_end = group_y_positions[-1] - big_bar_height / 2
            
            ax.plot([0, 0], [y_start, y_end], color='#34495e', linewidth=2, zorder=10, clip_on=False)
            
            for y_pos in group_y_positions:
                ax.plot([0, -tick_length], [y_pos, y_pos], color='#34495e',
                        linewidth=1.5, zorder=10, clip_on=False)
        
        current_idx += len(group)
    
    # 组间白色间隔
    for gap_start, gap_end in group_boundaries:
        rect = plt.Rectangle(
            (-6.8, gap_start - 2 * small_bar_height + 0.01),
            96.8, white_rect_height,
            facecolor='white', zorder=100, clip_on=False
        )
        ax.add_patch(rect)
    
    plt.title(f'Consensus Cluster {consensus_cluster_id}',
              pad=20, color='#2c3e50')
    
    plt.tight_layout(pad=0.5)
    output_path = os.path.join(output_dir, f'policy_vertical_bars_cluster_{consensus_cluster_id}.png')
    plt.savefig(output_path, dpi=300, bbox_inches='tight', facecolor='white', pad_inches=0.05)
    plt.close()

print("\n所有图表生成完成!")



正在生成共识聚类ID = 1 的图表...

正在生成共识聚类ID = 2 的图表...

正在生成共识聚类ID = 3 的图表...

正在生成共识聚类ID = 4 的图表...

正在生成共识聚类ID = 5 的图表...

正在生成共识聚类ID = 6 的图表...

正在生成共识聚类ID = 7 的图表...

所有图表生成完成!
